# Importing Essential Modules and Packages

In [1]:
import os
import pandas as pd
import librosa
import glob
import librosa.display
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.callbacks import EarlyStopping

from keras import regularizers

from sklearn.preprocessing import LabelEncoder

from datetime import datetime


ModuleNotFoundError: No module named 'librosa'

# Preparing the Data


In [2]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat")
cat_files = pd.DataFrame(filelist)
metadata = pd.read_csv("drive/MyDrive/ProjectMGN/CatandDog/train_test_split.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat'

In [3]:
filelist = os.listdir("drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog")
dog_files = pd.DataFrame(filelist)

FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog'

In [4]:
cat_files.head()

NameError: name 'cat_files' is not defined

In [5]:
dog_files.head()

NameError: name 'dog_files' is not defined

In [6]:
metadata.head()

NameError: name 'metadata' is not defined

In [7]:
cat_files

NameError: name 'cat_files' is not defined

# Merge the Cat Dog Dataset and Shuffling the Dataset

In [8]:
cat_files['label']='0'
dog_files['label']='1'
df = pd.concat([dog_files,cat_files],ignore_index=True)
df = df.rename(columns={0:'file'})
df = df.sample(frac=1).reset_index(drop=True)
df.head()

NameError: name 'cat_files' is not defined

In [9]:
# Checking Class Balance

df['label'].value_counts()

NameError: name 'df' is not defined

In [10]:
# Checking for Erroneous Folders

df[df['file']=='.DS_Store']

NameError: name 'df' is not defined

# Function to Extract Primary Features from Audio File

In [11]:
def extract_features(data):
  # file_name - Directory to cat/dog audio files
  try:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/cat/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  except FileNotFoundError:
    file_name = 'drive/MyDrive/ProjectMGN/CatandDog/cats_dogs/train/dog/'+str(data.file)
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  # print(file_name)
  mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
  stft = np.abs(librosa.stft(X))
  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
  tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
  label = data.label
  return mfccs, chroma, mel, contrast, tonnetz, label


# Generating Features


In [12]:
startTime=datetime.now()
features_labels = df.apply(extract_features, axis=1)
endtime=datetime.now()
print("Total Time Taken:",endtime-startTime)

NameError: name 'datetime' is not defined